#Модуль 20

##Открываю данные и разделяю их на контрольную и тестовую группы

In [ ]:
import pandas as pd
import missingno as msno
from scipy.stats import ttest_ind
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
df1 = pd.read_excel('/content/CASE_CLIENTS.xlsx')
df2 = pd.read_excel('/content/CASE_CONTRACTS.xlsx')
df3 = pd.read_excel('/content/CASE_LOSSES.xlsx')
df4 = pd.read_excel('/content/experiment_group.xlsx')

In [ ]:
df1.head()

,client_id,last_name,first_name,middle_name,age,sex
0,10000041307,З*****,Р*****,О*****,30,F
1,10000133158,И*****,М*****,Ю*****,37,M
2,100003773,Е*****,О*****,С*****,50,M
3,10000535718,Е*****,Ф*****,Ч*****,45,M
4,10000583119,И*****,Ф*****,Ф*****,28,M


In [ ]:
df1.shape

(3711, 6)

In [ ]:
df2.head()

,contract_id,contract_num,product_name,client_id,contract_status,currency_name,duration,country,price,insurance_amount
0,21111219,ТТЕ7227715*****,Страхование путешественников,10161870404,Действует,Российский рубль,10,Беларусь,1096,2000000
1,21111381,ТТЕ7227715*****,Страхование путешественников,1017159879,Действует,Российский рубль,7,Индонезия,1918,5000000
2,21112353,БАДАМСТЕ55*****,Страхование путешественников,10161883357,Действует,Российский рубль,10,Беларусь,1096,2000000
3,21113224,ТТЕ7227715*****,Страхование путешественников,100766461,Действует,Российский рубль,14,Грузия,1534,2000000
4,21113292,ТТЕ7227715*****,Страхование путешественников,10171604758,Действует,Российский рубль,7,Аргентина,1918,5000000


In [ ]:
df2.shape

(3711, 10)

In [ ]:
df3.head()

,loss_id,client_id,loss_name,loss_payout_amt
0,1,10000713193,Оказание медицинской помощи,100000
1,2,100014656,Оказание медицинской помощи,50000
2,3,10006545463,Оказание медицинской помощи,30000
3,4,10006611194,Оказание медицинской помощи,70000
4,5,1001091597,Оказание медицинской помощи,60000


In [ ]:
df3.shape

(45, 4)

In [ ]:
df4.head()

,client_id,group,experiment_id
0,10001718,control,policy_pricing
1,1017180395,control,policy_pricing
2,1017178047,control,policy_pricing
3,1017174614,control,policy_pricing
4,1017174071,control,policy_pricing


In [ ]:
df4.shape

(14842, 3)

In [ ]:
df = pd.merge(df1, df2, on='client_id')

In [ ]:
df = pd.merge(df, df3, how='left', on='client_id')

In [ ]:
df = pd.merge(df4, df, how='left', on='client_id')
df.shape

(14842, 20)

In [ ]:
df.head()

,client_id,group,experiment_id,last_name,first_name,middle_name,age,sex,contract_id,contract_num,product_name,contract_status,currency_name,duration,country,price,insurance_amount,loss_id,loss_name,loss_payout_amt
0,10001718,control,policy_pricing,Г*****,З*****,З*****,22.0,F,2.123569e+08,ТТЕ7227715*****,Страхование путешественников,Действует,Российский рубль,14.0,Египет,3836.0,5000000.0,NaN,NaN,NaN
1,1017180395,control,policy_pricing,Ж*****,Ю*****,Л*****,45.0,M,2.135234e+07,ДСЖ-5/1539*****,Страхование путешественников,Действует,Российский рубль,7.0,Таиланд,1918.0,5000000.0,NaN,NaN,NaN
2,1017178047,control,policy_pricing,А*****,Д*****,И*****,32.0,F,2.147454e+09,ТМБЕ579 № *****,Страхование путешественников,Действует,Российский рубль,14.0,Египет,3836.0,5000000.0,NaN,NaN,NaN
3,1017174614,control,policy_pricing,И*****,Ш*****,Р*****,35.0,F,2.135192e+07,ТМГ1535Е7 *****,Страхование путешественников,Действует,Российский рубль,7.0,Таиланд,1918.0,5000000.0,NaN,NaN,NaN
4,1017174071,control,policy_pricing,Ж*****,-*****,Н*****,61.0,F,2.162434e+08,ТТЕ7227777*****,Страхование путешественников,Завершен,Российский рубль,10.0,Канада,1096.0,2000000.0,NaN,NaN,NaN


In [ ]:
df_control = df[df['group'] == 'control']
df_test = df[df['group'] == 'test']
print(f'control:{df_control.shape}')
print(f'test:{df_test.shape}')

control:(6043, 20)
test:(6306, 20)


##Рассчитаю основные метрики:

###Конверсия в оформленность

In [ ]:
nan_count_test = df_test.contract_num.isna().sum()
nan_count_control = df_control.contract_num.isna().sum()

shape_test = df_test.shape[0]
shape_control = df_control.shape[0]

In [ ]:
reg_count_control = shape_test - nan_count_test
reg_count_test = shape_control - nan_count_control

In [ ]:
reg_CR_test = reg_count_control / shape_test
reg_CR_control =  reg_count_test / shape_control

In [ ]:
print(f'Конверия в оформленность на контрольной группе: {reg_CR_control}')
print(f'Конверия в оформленность на тестовой группе: {reg_CR_test}')

Конверия в оформленность на контрольной группе: 0.3225219261955982
Конверия в оформленность на тестовой группе: 0.26435141135426576


###Конверсия в выплату по страховке

In [ ]:
df_test.head()

,client_id,group,experiment_id,last_name,first_name,middle_name,age,sex,contract_id,contract_num,product_name,contract_status,currency_name,duration,country,price,insurance_amount,loss_id,loss_name,loss_payout_amt
1854,10111468947,test,policy_pricing,Ё*****,Б*****,Т*****,25.0,M,2.164933e+09,ТТЕ7227715*****,Страхование путешественников,Действует,Российский рубль,19.0,Турция,5205.0,5000000.0,NaN,NaN,NaN
1855,10191188103,test,policy_pricing,А*****,Ш*****,Е*****,56.0,M,2.153493e+07,ДСЖ-5/1539*****,Страхование путешественников,Действует,Российский рубль,14.0,Египет,3836.0,5000000.0,NaN,NaN,NaN
1856,10111378413,test,policy_pricing,Ж*****,И*****,В*****,33.0,M,2.164890e+09,ДСЖ-5/1539*****,Страхование путешественников,Действует,Российский рубль,18.0,Турция,4932.0,5000000.0,NaN,NaN,NaN
1857,10105565615,test,policy_pricing,В*****,Б*****,Б*****,33.0,F,2.121749e+08,ДСЖ-5/1539*****,Страхование путешественников,Действует,Российский рубль,10.0,Армения,1096.0,2000000.0,NaN,NaN,NaN
1858,10107579603,test,policy_pricing,З*****,П*****,Е*****,47.0,M,2.135595e+08,ТТЕ7227779*****,Страхование путешественников,Действует,Российский рубль,22.0,Турция,6027.0,5000000.0,NaN,NaN,NaN


In [ ]:
nan_loss_count_test = df_test.loss_id.isna().sum()
nan_loss_count_control = df_control.loss_id.isna().sum()

nan_count_test = df_test.contract_id.isna().sum()
nan_count_control = df_control.contract_id.isna().sum()

In [ ]:
loss_count_test = df_test.shape[0] - nan_loss_count_test
loss_count_control = df_control.shape[0] - nan_loss_count_control

contr_count_test = df_test.shape[0] - nan_count_test
contr_count_control = df_control.shape[0] - nan_count_control

In [ ]:
loss_CR_test = loss_count_test / contr_count_test
loss_CR_control = loss_count_control / contr_count_control

In [ ]:
print(f'Конверсия в убыток на контрольной группе: {loss_CR_control}')
print(f'Конверсия в убыток на тестовой группе: {loss_CR_test}')

Конверсия в убыток на контрольной группе: 0.01539250897896357
Конверсия в убыток на тестовой группе: 0.008398320335932814


###Средняя цена полиса

In [ ]:
df_control.head()

,client_id,group,experiment_id,last_name,first_name,middle_name,age,sex,contract_id,contract_num,product_name,contract_status,currency_name,duration,country,price,insurance_amount,loss_id,loss_name,loss_payout_amt
0,10001718,control,policy_pricing,Г*****,З*****,З*****,22.0,F,2.123569e+08,ТТЕ7227715*****,Страхование путешественников,Действует,Российский рубль,14.0,Египет,3836.0,5000000.0,NaN,NaN,NaN
1,1017180395,control,policy_pricing,Ж*****,Ю*****,Л*****,45.0,M,2.135234e+07,ДСЖ-5/1539*****,Страхование путешественников,Действует,Российский рубль,7.0,Таиланд,1918.0,5000000.0,NaN,NaN,NaN
2,1017178047,control,policy_pricing,А*****,Д*****,И*****,32.0,F,2.147454e+09,ТМБЕ579 № *****,Страхование путешественников,Действует,Российский рубль,14.0,Египет,3836.0,5000000.0,NaN,NaN,NaN
3,1017174614,control,policy_pricing,И*****,Ш*****,Р*****,35.0,F,2.135192e+07,ТМГ1535Е7 *****,Страхование путешественников,Действует,Российский рубль,7.0,Таиланд,1918.0,5000000.0,NaN,NaN,NaN
4,1017174071,control,policy_pricing,Ж*****,-*****,Н*****,61.0,F,2.162434e+08,ТТЕ7227777*****,Страхование путешественников,Завершен,Российский рубль,10.0,Канада,1096.0,2000000.0,NaN,NaN,NaN


In [ ]:
avg_price_control = df_control['price'].mean()
avg_price_test = df_test['price'].mean()

In [ ]:
print(f'Средняя цена полиса в контрольной группе: {avg_price_control}')
print(f'Средняя цена полиса в тестовой группе группе: {avg_price_test}')

Средняя цена полиса в контрольной группе: 3623.322216521293
Средняя цена полиса в тестовой группе группе: 4636.203359328134


##Провожу статистический анализ результатов теста

###Анализ конверсии в оформленность

Установим уровень значимости в 0.01

In [ ]:
successes = np.array([reg_count_control, reg_count_test])
nobs = np.array([shape_test, shape_control])

In [ ]:
print(f'stat:{proportions_ztest(successes, nobs)[0]}, p-value:{proportions_ztest(successes, nobs)[1]}')

stat:-7.101108398756141, p-value:1.2376025494738044e-12


P-value значительно ниже чем наш уровень значимости, следовательно отвергаем нулевую гипотезу и говорим о том, что разница между тестовой и контрольной группами статистически значима.

###Анализ конверсии в убыточность

Установим уровень значимости 0.05

In [ ]:
successes = np.array([loss_count_control, loss_count_test])
nobs = np.array([contr_count_control, contr_count_test])
print(f'stat:{proportions_ztest(successes, nobs)[0]}, p-value:{proportions_ztest(successes, nobs)[1]}')

stat:1.9122435674441198, p-value:0.05584496035189076


Как мы видим, p-value больше чем наш уровень значимости. Следовательно, мы не можем отвергать нулевую гипотезу и говорить о статистической разнице

###Анализ средней цены полиса

Установим уровень значимости 0.05

In [ ]:
ttest_ind(df_control['price'].dropna(), df_test['price'].dropna())

TtestResult(statistic=-3.1446351359597258, pvalue=0.0016764343314087742, df=3614.0)

P value ниже чем наш уровень значимости, следовательно мы отвергаем нулевую гипотезу и говорим о том что разница в метриках статистически значима

##Выводы и рекомендации

Из всего что выше, я могу сделать вывод о том, что в новой модели (тестовой группе) статистически значимо меньше конверсия в оформление, но, зато, больше средняя цена приобретенного страхования, и разница статистически значима. Конверсия в убыток же не дала статистически значимых результатов.

Я могу порекомендовать раскатить новую модель, так как пусть с ней и оформляют страховку реже, но зато оформляют ту, что дороже. Могу так же порекомендовать давать больше качественной долгой рекламы, в которой объясняется чем важна страховка, чтобы привлечь больше людей